In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import torch

In [ ]:
from thesis.data_management import SequenceManager
from thesis.flame import FlameHead

sm = SequenceManager(3)
flame_parmas = sm.flame_params[0:1]
flame_head = FlameHead()
v = flame_head.forward(flame_parmas)
v.shape

In [ ]:
from thesis.code_talker.models.code_talker_config import QuantizerConfig, QuantizationTrainingConfig
from thesis.code_talker.stage1_runner import Stage1Runner

config = QuantizerConfig()
training_config = QuantizationTrainingConfig()
runner = Stage1Runner(config, training_config)

In [ ]:
batch_size = 2
time = 101
input_dim = config.input_dim
x = torch.randn(batch_size, time, input_dim)
template = torch.randn(batch_size, input_dim)

quant, emb_loss, info = runner.model.encode(x)
print(quant.shape)  # (batch_size, z_quant_dim, time * face_qun) # (i.e. B, C, L*F)

dec = runner.model.decode(quant)
print(dec.shape)  # (batch_size, time, input_dim)
